<img src="http://www.ubu.es/sites/default/files/portal_page/images/logo_color_2l_dcha.jpg" height="150" width="150" align="right"/>

## Collaborative Filtering (2)
[Nacho Santos](www.nacho.santos.name)

## Import python packages

In [3]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [4]:
# Other functions necessary for this assignment
# The python file "recommender_system.py" must be in the same folder as this notebook, otherwise,
# you have to add the path to the file
from recommender_system import *

In [5]:
# This line is necessary to show matplotlib plots inside the jupyter notebook
%matplotlib inline

## 2 The cost funcion J and the gradient of J
The objective of this point is to build a function to compute the cost J and the corresponding gradient of J. In particular, you are going to implement a function called **cofiCostFunc()** with the arguments(inputs) and outputs detailed below (the code of the function is partially predefined in a cell right after).

**Arguments** (in this order)
* *parameters* (the paramaters of the cost function, i.e. X and $\theta$
* *Y* (matrix of ratings)
* *R* (matrix of watched movies)
* *n_users* (number of users)
* *n_movies* (number of movies)
* *n_characteristics* (number of the filter characteristics)
* *landa* (regularization parameter)

**Outputs** (in this order)
* *cost* (value of the cost function J)
* *gradient* (gradient of the cost function J)

In [11]:
# Cost and gradient function
def cofiCostFunc (parameters, Y, R, n_users, n_movies, n_features, lamb):
# parameters: vector with the matrices X and Theta foldes
# Y: matrix of ratings
# R: matrix of watched movies
# n_users: number of users (number of columns of the matrix Y)
# n_movies: number of movies (number of rows of the matrix Y)
# n_features: number of movies' features (a parameter of the CF algorithm)
# lamb: regularization term
#
# cofiCostFunc rerturns the cost J and the gradient of J

    # You need to return the following values correctly
    cost=0
    gradient=np.zeros_like(parameters)
    
    # Remember:
    #  (1) To unfold X and Theta from parameters before computing J and the gradients
    #  (2) To fold the gradient of J with respect to X and to Theta into a flattened vector gradient 
    #      that is returned by the function
    
    
    # YOUR CODE ..................................
    X = parameters[:n_movies*n_features].reshape(n_movies, n_features)
    Theta = parameters[n_movies*n_features:].reshape(n_users, n_features)
    cost=1/2*(np.sum(((np.dot(X, np.transpose(Theta))*R-Y)**2))+lamb*np.sum(Theta**2)+lamb*np.sum(X**2))
    
    Theta_gr=np.zeros((n_users, n_features))
    X_gr=np.zeros((n_movies, n_features))

    Theta_gr=np.dot(np.transpose(np.dot(X,np.transpose(Theta))*R-Y),X)+lamb*Theta
    X_gr=np.dot((np.dot(X, np.transpose(Theta))*R-Y), Theta)+lamb*X

    gradient=np.concatenate([X_gr.flatten(),Theta_gr.flatten()])
    
    
    
    
    

    # YOUR CODE (end) ..................................
    
    return (cost, gradient)

### 2.1 Input *parameters* of the cofiCostFunc

**Features X and preferences Theta**

The Collaborative Filtering (CF) algorithm is based on two sets of lineas regressions, the first one corresponds to the movies' features X, and the second one corresponds to the users' preferences Theta. Assuming n features, the matrix X will be:

$$X=\begin{bmatrix}x^{(1)}_{1} & ...& x^{(1)}_{n} \\. & ...& .\\x^{(m)}_{1} & ...& x^{(m)}_{n} \end{bmatrix}$$

where the i-th row of X corresponds to the feature vector $x^{(i)}$ for the i-th movie.

And the matrix Theta will be:

$$Theta=\Theta=\begin{bmatrix}\theta^{(1)}_{1} & ...& \theta^{(1)}_{n} \\. & ...& .\\\theta^{(u)}_{1} & ...& \theta^{(u)}_{n} \end{bmatrix}$$

where the j-th row of Theta corresponds to the preference vector $\theta^{(j)}$ for the j-th user. 

**Passing X and Theta to cofiCostFunc**

We are going to use a optimize package scipy.optimize that requires using a **flattened vector** of parameters. However, in our problem tha parameters to be optimized are represented by two matrices, i.e. X and Theta. So, X and Theta must be passed to the cofiCostFunc as a **(mxn)+(u+n) vector**, called **parameters**:

$${ \left[ \begin{matrix} { x }^{ (1) }, & ... & { x }^{ (m) }, \end{matrix}\begin{matrix} \theta ^{ (1) }, & ... & \theta ^{ (u) } \end{matrix} \right]  }_{ (m\cdot n)+(u\cdot n) }$$ 

However, inside the function, you can unfold the vector **parameters** and build the matrices X and Theta to compute J and the gradients according to the equations explained in class.

### 2.2 Computing the cost J
Suppose that the vector of features $x^{(i)}$ of the film i and the vector of preferences $\theta^{(j)}$ of the user j are known, then the **estimate of the rating** of the user j for the movie i will be:

$$\widehat{y}^{(i,j)}=x^{(i)}(\theta^{(j)})^{T}$$

The error of the estimate will be the difference between the estimate of rating $\widehat{y}^{(i,j)}$ and the real ratings $y^{(i,j)}$

The **cost J** is defined as the the average of the squares of the errors plus two regularization terms:

$$J=\frac { 1 }{ 2 } \sum _{ (i,j):r(i,j)=1 }^{  }{ \left( { x }^{ (i) }({ \theta  }^{ (j) })^{ T }-{ y }^{ (i,j) } \right) ^{ 2 } } +\quad \frac { \lambda  }{ 2 } \sum _{ i=1 }^{ m }{ \sum _{ k=1 }^{ n }{ ({ x }_{ k }^{ (i) })^{ 2 } }  } +\frac { \lambda  }{ 2 } \sum _{ j=1 }^{ u }{ \sum _{ k=1 }^{ n }{ ({ \theta  }_{ k }^{ (j) })^{ 2 } }  } $$


### Task 7
***
Implement the cost J as a vectorized expression (recommended). For example, the estimate of ratings can be expressed as:

$$\widehat{Y}=X\Theta^{T}$$

Now, go back and **complete the cofiCostFunc code to compute the cost J**. Remeber that J is scalar value.

### 2.3 Checking the cost J
Now, you will import a data set and check the cofiCostFunc.

In [7]:
# load dataset for checking
Y=np.load('YmatrixTest.npy')
R=np.load('RmatrixTest.npy')
X=np.load('XmatrixTest.npy')
Theta=np.load('ThetamatrixTest.npy')

# dimension
n_users = Y.shape[1]
n_movies = Y.shape[0]
n_features = X.shape[1]

### Task 8
***
Call cofiCostFunc with lamb=0 (without regularization term) and check the result

In [12]:
# Evaluate Cost J (without regularization term)
J=0
parameters=np.concatenate([X.flatten(),Theta.flatten()])

# YOUR CODE ..................................
# call cofiCostFunc with lamb=0 (without regularization term)
J=cofiCostFunc(parameters, Y, R, n_users, n_movies, n_features, 0)[0]




# YOUR CODE (end)..................................

print('The value of J (without regularization term) is %0.2f (it should be 22.22)' % J )

The value of J (without regularization term) is 22.22 (it should be 22.22)


### Task 9
***
Call cofiCostFunc with lamb=1.5 (with regularization term) and check the result

In [13]:
# Evaluate Cost J (with regularization term)
J=0
parameters=[]

# YOUR CODE ..................................
# call cofiCostFunc with lamb=1.5 (without regularization term)
parameters=np.concatenate([X.flatten(),Theta.flatten()])
J=cofiCostFunc(parameters, Y, R, n_users, n_movies, n_features, 1.5)[0]





# YOUR CODE (end)..................................

print('The value of J (with regularization term equal to 1.5) is %0.2f (it should be 31.34)' % J )

The value of J (with regularization term equal to 1.5) is 31.34 (it should be 31.34)


### 2.4 Computing the gradient of J
The **gradient of J** depends on the two types of parameters, i.e. X and Theta. The corresponding equations are:

$$\frac { \partial J }{ \partial { \theta  }_{ k }^{ (j) } } =\sum _{ i:r(i,j)=1 }^{  }{ \left( { x }^{ (i) }({ \theta  }^{ (j) })^{ T }-{ y }^{ (i,j) } \right) { x }_{ k }^{ (i) } } +\lambda { \theta  }_{ k }^{ (j) }$$

$$\frac { \partial J }{ \partial { x }_{ k }^{ (i) } } =\sum _{ j:r(i,j)=1 }^{  }{ \left( { x }^{ (i) }({ \theta  }^{ (j) })^{ T }-{ y }^{ (i,j) } \right) \theta _{ k }^{ (j) } } +\lambda { x }_{ k }^{ (i) }$$

### Task 10
***
Now, go back and **complete the cofiCostFunc code to compute the gradient of J**. Remember to use vectorized operations instead of using for loops.

Note that the outputs of cofiCostFunc are the cost J (scalar value) and the gradient, again a **flattened vector of the corresponding gradients of X and Theta**:

$${ \left[ \begin{matrix} \frac { \partial J }{ \partial { x }^{ (1) } } , & ... & \frac { \partial J }{ \partial { x }^{ (m) } } , & \frac { \partial J }{ \partial \theta ^{ (1) } } , & ... & \frac { \partial J }{ \partial \theta ^{ (u) } }  \end{matrix} \right]  }_{ (m\cdot n)+(u\cdot n) }$$

After computing both gradients, you should reshape them into a flattened vector called **gradient** that will be returned by the cofiCostFunc.

### 2.5 Checking the gradient of J
For the same dataset of the last poit, you will check the gradient of J computed by your cofiCostFunc

In [14]:
# Check gradients (without regularization term) by running the next function
checkCostFunction(cofiCostFunc,0)

The above two columns you get should be very similar.
(Left - Your Numerical Gradient, Right - Analytical Gradient)

(0.6510065716669455, 0.6510065716673111)
(0.8086199481496803, 0.8086199481496615)
(0.1748463082038021, 0.17484630820398653)
(-0.02083443377765004, -0.020834433777354423)
(-0.0969136727230202, -0.09691367272315933)
(-0.05519967229195011, -0.05519967229209706)
(-0.142760283869392, -0.14276028386938608)
(-0.17732357608102323, -0.17732357608102228)
(-0.03834232967708795, -0.0383423296769289)
(0.03836026308018381, 0.03836026308007702)
(0.13816100211144766, 0.13816100211155571)
(-0.12240560103710107, -0.12240560103713898)
(0.7411323881362231, 0.7411323881372027)
(0.5063526181378619, 0.5063526181385095)
(0.6019881590632603, 0.6019881590637675)
(-0.034381790864623785, -0.03438179086482943)
(-0.03184351007767816, -0.03184351007746739)
(-0.02758451165874032, -0.027584511658768275)
(-0.042736615147864754, -0.042736615147863505)
(-0.03958152850397356, -0.03958152850408796)
(-0.03428

In [15]:
# Check gradients (with regularization term) by running the next function
checkCostFunction(cofiCostFunc,1.5)

The above two columns you get should be very similar.
(Left - Your Numerical Gradient, Right - Analytical Gradient)

(-0.8438778121666601, -0.8438778121611398)
(-0.4938559369138673, -0.4938559369085195)
(0.19806508614550467, 0.19806508614383111)
(1.4376776945423941, 1.4376776945371665)
(0.2985279134515295, 0.29852791345101465)
(0.8378962250832345, 0.837896225079484)
(1.5759434813045203, 1.5759434813026372)
(1.647944208378327, 1.6479442083797882)
(1.7928284798252037, 1.7928284798243164)
(0.9970991699148612, 0.9970991699117586)
(0.4381528012986635, 0.4381528012980987)
(0.48982028583388626, 0.48982028583605)
(1.3602860015016205, 1.3602860015027831)
(1.207720825884273, 1.207720825884889)
(0.8066626286451495, 0.8066626286398169)
(0.7645504397135738, 0.7645504397112362)
(1.5899860724033488, 1.5899860724011465)
(1.249985418216859, 1.249985418214183)
(0.727836136102944, 0.7278361361022266)
(0.5309091039107372, 0.5309091039126718)
(-0.11300006953263164, -0.11300006953249297)
(0.6697664570332051

## 3 Learning and recommendation
Finally, you will use your cofiCostFun to make predictions using the initial Movielens dataset. Part of the python code you need is already written in the next cells. You only have to complete those lines that are explicitly required.

### Task 11
***
Load the matrix Y and R computed in the first notebook.

In [19]:
# Task: load matrix Y and R
# YOUR CODE ..................................
Y=np.load("Y.npy")
R=np.load("R.npy")




### Task 12
***
* Get the number of users and movies and assign the corresponding variables n_users, n_movies.
* Set the initial parameters (Theta, X) with random values.
* Fold X and Theta into the variable initial_parameters.

In [17]:
# Set the number of features
n_features = 100

In [21]:
# YOUR CODE ..................................
n_movies=Y.shape[0]
n_users=Y.shape[1]
X = np.random.rand(n_movies,n_features)
Theta= np.random.rand(n_users,n_features)
initial_parameters=np.zeros_like(parameters)
initial_parameters=np.concatenate([X.flatten(),Theta.flatten()])


Now, we set the rest of the parameters and minimize the function

In [22]:
# Set the regularization parameter
lamb = 10

# Define a function to be minimized
def cofiCostFunc_minimize(parameters):
    return cofiCostFunc(parameters,Y, R, n_users, n_movies, n_features,lamb)

# Set the number of iteations
max_iter=200

In [23]:
# Minimize the function using minimize from the package scipy.optimize and get the optimized parameters
parameters = (minimize(cofiCostFunc_minimize,initial_parameters,method="CG",jac=True,
                   options={'maxiter':max_iter, "disp":True})).x

         Current function value: 66581.227178
         Iterations: 200
         Function evaluations: 305
         Gradient evaluations: 305


c:\Users\jorge\OneDrive\Documentos\4º Curos\gi\ml-100k\venv\Lib\site-packages\scipy\optimize\_minimize.py:703: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_cg(fun, x0, args, jac, callback, **options)


### Task 13
***
Get the matrix of predictions P


In [41]:
# YOUR CODE ..................................
X = parameters[:n_movies*n_features].reshape(n_movies, n_features)
Theta = parameters[n_movies*n_features:].reshape(n_users, n_features)
P=np.zeros((n_movies,n_users))
P=np.dot(X,np.transpose(Theta))






### Task 14
***
Show the titles of the top-5 predictions for the first user u=0, for those films user u did not watch: r(i,u)=0 (they will be the top-5 recommendations)

#### Tips
* You can import movies' titles using Pandas (see the first notebook)


In [43]:
# YOUR CODE ..................................
from pandas import read_table
# read csv file
items = read_table('u.item',header=None,sep='|',encoding='ISO-8859-1')
# remove collumns 2-24
items.drop(range(2,24),axis=1, inplace=True)
print(P)

top5= np.argsort(P[R[:,0]==0,0])[::-1][:5]

for i in top5:
    
    print(items[1][i])
    print(P[i][0])






[[4.5071559  3.88225438 2.91447481 ... 4.28054535 4.45649066 3.39094925]
 [2.99855559 2.65958974 1.89421228 ... 2.68327533 3.64366172 3.86049875]
 [3.56991802 2.28718326 2.05855209 ... 2.48468672 3.14679429 2.74344322]
 ...
 [0.4343731  0.32051774 0.28044773 ... 0.37123644 0.35461338 0.41778969]
 [0.46900636 0.4845817  0.26239095 ... 0.42878835 0.63049566 0.54955072]
 [0.8905475  0.62655483 0.44370646 ... 0.58564829 0.68099479 0.66258465]]
Belle de jour (1967)
3.556772149762193
Sneakers (1992)
3.7653958019628586
Get Shorty (1995)
3.671290590030943
Mr. Smith Goes to Washington (1939)
3.7055028865986914
Mighty Aphrodite (1995)
4.416987019388705
